Funnel na checkout w tabeli `GA360`

In [ ]:
%%bigquery

with addToCart as (
    select
    date,
    format_date('%Y%U',parse_date("%Y%m%d",date)) as week_of_year,
    format_date('%Y%m',parse_date("%Y%m%d",date)) as month_of_year,
    case when device.deviceCategory like 'mobile' then 'mobile'
    else 'desktop' end device,
    case when (SELECT value FROM hits.customDimensions WHERE index=2) in ('true') then true else false end moj_empik, # gdzie true to klient z mój empik [uzytkownik]
    case when (SELECT value FROM hits.customDimensions WHERE index=22) in ('true') then true else false end empik_premium, # gdzie true to klient premium [uzytkownik]
    case when ((SELECT value FROM hits.customDimensions WHERE index=2) in ('true') AND (SELECT value FROM hits.customDimensions WHERE index=22) in ('false')) then true else false end empik_premium_free, # [uzytkownik]
    case when (SELECT value FROM hits.customDimensions WHERE index=51) in ('Zalogowany') then true else false end user_status, # gdzie true to klient zalogowany [uzytkownik]
    CONCAT(fullVisitorId,visitId) AS total_visits,
    fullVisitorId total_user,
    totals.newVisits new_visits,
    hits.eventInfo.eventValue
FROM
    `empik-ga360.304756.ga_sessions_*`,
    UNNEST(hits) AS hits
WHERE
    eventInfo.eventCategory = 'Ecommerce'
    AND eventInfo.eventAction = 'Add to Cart'
    and _table_suffix between '20211119' 
    and format_date('%Y%m%d',date_sub(current_date(), interval 1 day))
GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12
),

checkoutStep1 as (
    select
    date,
    format_date('%Y%U',parse_date("%Y%m%d",date)) as week_of_year,
    format_date('%Y%m',parse_date("%Y%m%d",date)) as month_of_year,
    case when device.deviceCategory like 'mobile' then 'mobile'
    else 'desktop' end device,
    case when (SELECT value FROM hits.customDimensions WHERE index=2) in ('true') then true else false end moj_empik, 
    case when (SELECT value FROM hits.customDimensions WHERE index=22) in ('true') then true else false end empik_premium, 
    case when ((SELECT value FROM hits.customDimensions WHERE index=2) in ('true') AND (SELECT value FROM hits.customDimensions WHERE index=22) in ('false')) then true else false end empik_premium_free, 
    case when (SELECT value FROM hits.customDimensions WHERE index=51) in ('Zalogowany') then true else false end user_status, 
    CONCAT(fullVisitorId,visitId) AS total_visits,
    fullVisitorId total_user,
    totals.newVisits new_visits,
    hits.eventInfo.eventValue
FROM
    `empik-ga360.304756.ga_sessions_*`,
    UNNEST(hits) AS hits,
    UNNEST(product) AS product 
WHERE
    eventInfo.eventCategory = 'Ecommerce'
    AND eventInfo.eventAction = 'Checkout step 1'
    and REGEXP_CONTAINS(page.pagePath, 'www.empik.com/cart/index.jsp') = true
    and _table_suffix between '20211119' 
    and format_date('%Y%m%d',date_sub(current_date(), interval 1 day))
GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12
),

checkoutStep2 as (
    select
    date,
    format_date('%Y%U',parse_date("%Y%m%d",date)) as week_of_year,
    format_date('%Y%m',parse_date("%Y%m%d",date)) as month_of_year,
    case when device.deviceCategory like 'mobile' then 'mobile'
    else 'desktop' end device,
    case when (SELECT value FROM hits.customDimensions WHERE index=2) in ('true') then true else false end moj_empik, 
    case when (SELECT value FROM hits.customDimensions WHERE index=22) in ('true') then true else false end empik_premium, 
    case when ((SELECT value FROM hits.customDimensions WHERE index=2) in ('true') AND (SELECT value FROM hits.customDimensions WHERE index=22) in ('false')) then true else false end empik_premium_free, 
    case when (SELECT value FROM hits.customDimensions WHERE index=51) in ('Zalogowany') then true else false end user_status, 
    CONCAT(fullVisitorId,visitId) AS total_visits,
    fullVisitorId total_user,
    totals.newVisits new_visits,
FROM
    `empik-ga360.304756.ga_sessions_*`,
    UNNEST(hits) AS hits
WHERE
    eventInfo.eventCategory = 'Funnel'
    AND eventInfo.eventAction = 'checkoutStep2'
    and _table_suffix between '20211119' 
    and format_date('%Y%m%d',date_sub(current_date(), interval 1 day))
GROUP BY 1,2,3,4,5,6,7,8,9,10,11
),

checkoutStep3 as (
    select
    date,
    format_date('%Y%U',parse_date("%Y%m%d",date)) as week_of_year,
    format_date('%Y%m',parse_date("%Y%m%d",date)) as month_of_year,
    case when device.deviceCategory like 'mobile' then 'mobile'
    else 'desktop' end device,
    case when (SELECT value FROM hits.customDimensions WHERE index=2) in ('true') then true else false end moj_empik, 
    case when (SELECT value FROM hits.customDimensions WHERE index=22) in ('true') then true else false end empik_premium, 
    case when ((SELECT value FROM hits.customDimensions WHERE index=2) in ('true') AND (SELECT value FROM hits.customDimensions WHERE index=22) in ('false')) then true else false end empik_premium_free, 
    case when (SELECT value FROM hits.customDimensions WHERE index=51) in ('Zalogowany') then true else false end user_status, 
    CONCAT(fullVisitorId,visitId) AS total_visits,
    fullVisitorId total_user,
    totals.newVisits new_visits
FROM
    `empik-ga360.304756.ga_sessions_*`,
    UNNEST(hits) AS hits
WHERE
    eventInfo.eventCategory = 'Funnel'
    AND eventInfo.eventAction = 'checkoutStep3'
    and _table_suffix between '20211119' 
    and format_date('%Y%m%d',date_sub(current_date(), interval 1 day))
GROUP BY 1,2,3,4,5,6,7,8,9,10,11
),

purchase as (
    select
    date,
    format_date('%Y%U',parse_date("%Y%m%d",date)) as week_of_year,
    format_date('%Y%m',parse_date("%Y%m%d",date)) as month_of_year,
    case when device.deviceCategory like 'mobile' then 'mobile'
    else 'desktop' end device,
    case when (SELECT value FROM hits.customDimensions WHERE index=2) in ('true') then true else false end moj_empik, 
    case when (SELECT value FROM hits.customDimensions WHERE index=22) in ('true') then true else false end empik_premium, 
    case when ((SELECT value FROM hits.customDimensions WHERE index=2) in ('true') AND (SELECT value FROM hits.customDimensions WHERE index=22) in ('false')) then true else false end empik_premium_free, 
    case when (SELECT value FROM hits.customDimensions WHERE index=51) in ('Zalogowany') then true else false end user_status, 
    CONCAT(fullVisitorId,visitId) AS total_visits,
    fullVisitorId total_user,
    totals.newVisits new_visits,
    hits.eventInfo.eventValue
FROM
    `empik-ga360.304756.ga_sessions_*`,
    UNNEST(hits) AS hits
WHERE
    eventInfo.eventCategory = 'Ecommerce'
    AND eventInfo.eventAction = 'Purchase'
    and _table_suffix between '20211119' 
    and format_date('%Y%m%d',date_sub(current_date(), interval 1 day))
GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12
),

checkoutStep4 as (
    select
    date,
    format_date('%Y%U',parse_date("%Y%m%d",date)) as week_of_year,
    format_date('%Y%m',parse_date("%Y%m%d",date)) as month_of_year,
    case when device.deviceCategory like 'mobile' then 'mobile'
    else 'desktop' end device,
    case when (SELECT value FROM hits.customDimensions WHERE index=2) in ('true') then true else false end moj_empik, 
    case when (SELECT value FROM hits.customDimensions WHERE index=22) in ('true') then true else false end empik_premium, 
    case when ((SELECT value FROM hits.customDimensions WHERE index=2) in ('true') AND (SELECT value FROM hits.customDimensions WHERE index=22) in ('false')) then true else false end empik_premium_free,
    case when (SELECT value FROM hits.customDimensions WHERE index=51) in ('Zalogowany') then true else false end user_status, 
    CONCAT(fullVisitorId,visitId) AS total_visits,
    fullVisitorId total_user,
    totals.newVisits new_visits
FROM
    `empik-ga360.304756.ga_sessions_*`,
    UNNEST(hits) AS hits
WHERE
    eventInfo.eventCategory = 'Funnel'
    AND eventInfo.eventAction = 'checkoutStep4'
    and _table_suffix between '20211119' 
    and format_date('%Y%m%d',date_sub(current_date(), interval 1 day))
GROUP BY 1,2,3,4,5,6,7,8,9,10,11  
)

select
atc.date,
atc.week_of_year,
atc.month_of_year,
atc.device,
atc.empik_premium empik_premium,
atc.empik_premium_free,
atc.user_status, 
atc.new_visits,
-- addToCart
count(distinct atc.total_visits) addToCart_total_visits,
count(distinct atc.total_user) addToCart_total_user,
sum(atc.eventValue) addToCart_event_value,
-- checkoutStep1
count(distinct cs1.total_visits) checkoutStep1_total_visits,
count(distinct cs1.total_user) checkoutStep1_total_user,
sum(cs1.eventValue) checkoutStep1_event_value,
-- checkoutStep2
count(distinct cs2.total_visits) checkoutStep2_total_visits,
count(distinct cs2.total_user) checkoutStep2_total_user,
-- checkoutStep3
count(distinct cs3.total_visits) checkoutStep3_total_visits,
count(distinct cs3.total_user) checkoutStep3_total_user,
-- purchase
count(distinct p.total_visits) purchase_total_visits,
count(distinct p.total_user) purchase_total_user,
sum(p.eventValue) purchase_event_value,
-- checkoutStep4
count(distinct cs4.total_visits) checkoutStep4_total_visits,
count(distinct cs4.total_user) checkoutStep4_total_user

from addToCart atc LEFT JOIN checkoutStep1 cs1 on atc.date = cs1.date and atc.total_visits = cs1.total_visits
LEFT JOIN checkoutStep2 cs2 on cs2.date = cs1.date and cs2.total_visits = cs1.total_visits
LEFT JOIN checkoutStep3 cs3 on cs3.date = cs2.date and cs3.total_visits = cs2.total_visits
LEFT JOIN purchase p on p.date = cs3.date and p.total_visits = cs3.total_visits
LEFT JOIN checkoutStep4 cs4 on cs4.date = p.date and cs4.total_visits = p.total_visits
group by 1,2,3,4,5,6,7,8